# Bioinformatics Genome Analysis: Gene Finding and Prediction

This notebook demonstrates fundamental methods for working with genomic data:
1. Loading and parsing genomic files (FASTA, GFF)
2. Finding open reading frames (ORF)
3. Comparing predicted genes with reference data
4. Evaluating prediction quality

## PART 1: LOADING E.COLI DATA

Loading the reference genome of Escherichia coli strain K-12 - one of the most well-studied bacterial genomes.

In [ ]:
print("=== Loading E.coli genome ===")
print("Downloading reference genome of Escherichia coli strain K-12")
print("This is one of the most well-studied bacterial genomes")

# Download compressed E.coli genome
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz

In [ ]:
# Check downloaded file size
!ls -hlrt

print("\nDecompressing genome...")
# Decompress archive
!gzip -d GCF_000005845.2_ASM584v2_genomic.fna.gz

# Check decompressed file size
!ls -hlrt

In [ ]:
print("\nViewing file header:")
print("FASTA format contains:")
print("- Header starting with '>'")
print("- DNA sequence in standard symbols (A, T, G, C)")

# View first 10 lines of genome
!head GCF_000005845.2_ASM584v2_genomic.fna

## PART 2: PARSING FASTA FILE

Creating a function to read FASTA format and load genomic sequence.

In [ ]:
print("\n=== Parsing FASTA file ===")
print("Creating function to read FASTA format")

def parse_fasta(filepath):
    """
    Parses FASTA file and returns pairs (header, sequence)
    
    FASTA format:
    >Sequence_header
    ATGCGATCGATCG...
    ATCGATCGATCGA...
    
    Args:
        filepath: path to FASTA file
    
    Yields:
        tuple: (header, sequence)
    """
    header = None
    sequence_lines = []

    with open(filepath, 'r') as file:
        for line in file:
            line = line.strip()  # Remove spaces and newline characters
            if not line:
                continue  # Skip empty lines
            if line.startswith('>'):
                if header is not None:
                    # If header exists, return previous sequence
                    yield header, ''.join(sequence_lines)
                header = line[1:]  # Remove '>' at beginning
                sequence_lines = []  # Reset buffer for new sequence
            else:
                sequence_lines.append(line)

        # Return last sequence in file
        if header is not None:
            yield header, ''.join(sequence_lines)

In [ ]:
# Apply parser to our file
fasta_path = "GCF_000005845.2_ASM584v2_genomic.fna"

print("Analyzing FASTA file structure:")
for header, sequence in parse_fasta(fasta_path):
    print("Header:", header)
    print("Sequence length:", len(sequence), "nucleotides")
    print("First 60 characters:", sequence[:60] + '...')
    
    # Save sequence for further analysis
    genome_sequence = sequence
    break  # E.coli has one chromosome, so take the first

print(f"\nComplete E.coli genome length: {len(genome_sequence):,} nucleotides")

## PART 3: BASIC DNA OPERATIONS

DNA is double-stranded, and genes can be located on either strand. To analyze the opposite strand, we need to compute the reverse complement sequence.

In [ ]:
print("\n=== Basic DNA operations ===")

def reverse_complement(dna_seq):
    """
    Computes reverse complement DNA sequence
    
    DNA is double-stranded, and genes can be on either strand.
    To analyze the opposite strand we need to:
    1. Replace each nucleotide with its complement (A↔T, G↔C)
    2. Reverse the sequence (read right to left)
    
    Args:
        dna_seq: DNA sequence
    
    Returns:
        str: reverse complement sequence
    """
    complement = {
        'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G',
        'a': 't', 't': 'a', 'g': 'c', 'c': 'g',
        'N': 'N', 'n': 'n'  # N denotes undefined nucleotide
    }
    reversed_seq = dna_seq[::-1]  # Reverse sequence
    rev_comp = ''.join(complement.get(base, base) for base in reversed_seq)
    return rev_comp

# Demonstrate function
test_seq = "ATGCGATCG"
print(f"Original sequence:      {test_seq}")
print(f"Reverse complement:     {reverse_complement(test_seq)}")

print(f"\nGenome length: {len(genome_sequence):,} bp")

# Create reverse complement sequence for minus strand analysis
rev_comp_sequence = reverse_complement(genome_sequence)
print("Reverse complement sequence created")

## PART 4: FINDING START CODONS

In bacteria, genes usually start with the start codon ATG (methionine). Let's analyze their distribution.

In [ ]:
print("\n=== Start codon analysis ===")
print("In bacteria, genes usually start with start codon ATG (methionine)")

# Count start codons on both strands
methionine_start = "ATG"
methionine_start_rc = "CAT"  # ATG on reverse strand appears as CAT on forward

hits_plus = genome_sequence.count(methionine_start)
hits_minus = genome_sequence.count(methionine_start_rc)

print(f"Start codons ATG on plus strand:  {hits_plus:,}")
print(f"Start codons ATG on minus strand: {hits_minus:,}")
print(f"Total ATG count:                  {hits_plus + hits_minus:,}")

print(f"\nATG frequency: {(hits_plus + hits_minus) / len(genome_sequence) * 1000:.2f} per 1000 bp")
print("This roughly corresponds to random distribution (expected ATG frequency = 1/64 ≈ 0.016)")

## PART 5: LOADING GENE ANNOTATION (GFF)

GFF (General Feature Format) contains gene coordinates and descriptions - reference data for comparison.

In [ ]:
print("\n=== Loading gene annotation ===")
print("GFF (General Feature Format) contains gene coordinates and descriptions")

# Download GFF file with E.coli annotation
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gff.gz

# Decompress
!gzip -d GCF_000005845.2_ASM584v2_genomic.gff.gz

In [ ]:
print("\nViewing GFF file structure:")
print("GFF contains 9 columns:")
print("1. Chromosome/contig")
print("2. Annotation source") 
print("3. Feature type (gene, CDS, etc.)")
print("4. Start (1-based)")
print("5. End (inclusive)")
print("6. Score")
print("7. Strand (+/-)")
print("8. Phase")
print("9. Attributes")

# Show first lines of GFF
!head GCF_000005845.2_ASM584v2_genomic.gff

## PART 6: PARSING GFF FILE

Creating a function to extract gene information from GFF file.

In [ ]:
print("\n=== Parsing GFF file ===")

def parse_gff(filepath):
    """
    Parses GFF file and extracts genetic element information
    
    Args:
        filepath: path to GFF file
    
    Returns:
        list: list of dictionaries with information about each element
    """
    annotations = []
    with open(filepath, 'r') as file:
        for line in file:
            line = line.strip()
            if not line or line.startswith('#'):
                continue  # Skip comments and empty lines

            parts = line.split('\t')
            if len(parts) != 9:
                continue  # Skip incorrect lines

            seqid, source, feature_type, start, end, score, strand, phase, attributes = parts

            # Parse attributes column into dictionary
            attr_dict = {}
            for attr in attributes.split(';'):
                if '=' in attr:
                    key, value = attr.split('=', 1)
                    attr_dict[key] = value

            annotations.append({
                'seqid': seqid,
                'source': source,
                'type': feature_type,
                'start': int(start),
                'end': int(end),
                'score': score if score != '.' else None,
                'strand': strand,
                'phase': phase if phase != '.' else None,
                'attributes': attr_dict
            })

    return annotations

# Parse GFF file
print("Parsing GFF file...")
annotations = parse_gff("GCF_000005845.2_ASM584v2_genomic.gff")
print(f"Loaded {len(annotations)} annotation records")

# Analyze element types
from collections import Counter
feature_types = Counter([ann['type'] for ann in annotations])
print("\nGenetic element types:")
for feature_type, count in feature_types.most_common():
    print(f"  {feature_type}: {count}")

## PART 7: GENE LENGTH ANALYSIS

Building gene length distribution and finding extreme values.

In [ ]:
print("\n=== Gene length analysis ===")

import matplotlib.pyplot as plt

def plot_gene_lengths(gff_file):
    """
    Plots histogram of gene length distribution and finds extreme values
    """
    features = parse_gff(gff_file)
    genes = [f for f in features if f['type'] == 'gene']
    gene_lengths = [gene['end'] - gene['start'] + 1 for gene in genes]

    if not gene_lengths:
        print("No genes found in GFF file")
        return

    # Find shortest and longest genes
    min_length = min(gene_lengths)
    max_length = max(gene_lengths)
    avg_length = sum(gene_lengths) / len(gene_lengths)

    min_index = gene_lengths.index(min_length)
    max_index = gene_lengths.index(max_length)

    # Plot histogram
    plt.figure(figsize=(12, 6))
    plt.hist(gene_lengths, bins=50, edgecolor='black', alpha=0.7)
    plt.title("E.coli Gene Length Distribution")
    plt.xlabel("Gene length (nucleotides)")
    plt.ylabel("Number of genes")
    plt.axvline(avg_length, color='red', linestyle='--', label=f'Average length: {avg_length:.0f} bp')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Print statistics
    print(f"Total number of genes: {len(genes)}")
    print(f"Shortest gene: {min_length} bp")
    print(f"Longest gene: {max_length} bp")
    print(f"Average gene length: {avg_length:.1f} bp")
    
    # Information about extreme genes
    min_gene = genes[min_index]
    max_gene = genes[max_index]
    
    print(f"\nShortest gene:")
    print(f"  ID: {min_gene['attributes'].get('ID', 'unknown')}")
    print(f"  Position: {min_gene['start']}-{min_gene['end']}")
    
    print(f"\nLongest gene:")
    print(f"  ID: {max_gene['attributes'].get('ID', 'unknown')}")
    print(f"  Position: {max_gene['start']}-{max_gene['end']}")

# Analyze E.coli gene lengths
gff_path = "GCF_000005845.2_ASM584v2_genomic.gff"
plot_gene_lengths(gff_path)

## PART 8: FINDING OPEN READING FRAMES (ORF)

ORF (Open Reading Frame) - a DNA region that potentially codes for a protein.

**ORF criteria:**
1. Starts with start codon (ATG)
2. Ends with stop codon (TAA, TAG, TGA)
3. Is in one reading frame (multiple of 3 nucleotides)
4. Contains no intermediate stop codons

**DNA has 6 possible reading frames:**
- 3 on forward strand (starting at positions 0, 1, 2)
- 3 on reverse strand

In [ ]:
def find_orfs_bacterial(dna_sequence: str) -> list:
    """
    Finds open reading frames (ORFs) in genomic sequence.
    
    Algorithm:
    1. Find all ATG codons in each of 3 reading frames
    2. For each ATG find nearest stop codon in same frame
    3. Check that ORF is not nested in already found ORF of same frame
    
    Args:
        dna_sequence: DNA sequence
    
    Returns:
        list: list of dictionaries with ORF information
    """
    if not dna_sequence:
        return []

    dna_sequence = dna_sequence.upper()
    n = len(dna_sequence)

    start_codon = "ATG"
    stop_codons = ["TAA", "TAG", "TGA"]

    found_orfs = []
    
    # Track found ORFs for each frame
    identified_orf_regions_by_frame = {0: [], 1: [], 2: []}

    # Analyze 3 reading frames
    for frame_offset in range(3):
        print(f"  Analyzing frame {frame_offset + 1}...")
        
        # Find all ATG in current frame
        potential_atg_indices = []
        for i in range(frame_offset, n - 2, 3):
            codon = dna_sequence[i : i + 3]
            if codon == start_codon:
                potential_atg_indices.append(i)

        print(f"    Found {len(potential_atg_indices)} potential start codons")

        # For each ATG find ORF
        valid_orfs_in_frame = 0
        for atg_start_index in potential_atg_indices:
            # Check if this ATG is nested in already found ORF
            is_nested = False
            for orf_start, orf_end in identified_orf_regions_by_frame[frame_offset]:
                if orf_start <= atg_start_index < orf_end:
                    is_nested = True
                    break

            if is_nested:
                continue

            # Find first stop codon after ATG
            for j in range(atg_start_index + 3, n - 2, 3):
                codon = dna_sequence[j : j + 3]
                if codon in stop_codons:
                    orf_end_index = j + 3
                    orf_sequence = dna_sequence[atg_start_index:orf_end_index]
                    
                    orf_info = {
                        "start": atg_start_index,
                        "end": orf_end_index,
                        "frame": frame_offset + 1,
                        "sequence": orf_sequence,
                        "length": len(orf_sequence)
                    }
                    found_orfs.append(orf_info)
                    
                    # Remember this ORF region
                    identified_orf_regions_by_frame[frame_offset].append(
                        (atg_start_index, orf_end_index)
                    )
                    valid_orfs_in_frame += 1
                    break

        print(f"    Found {valid_orfs_in_frame} valid ORFs")

    # Sort by position
    found_orfs.sort(key=lambda x: (x['start'], x['frame']))
    return found_orfs

In [ ]:
# Find ORFs on both strands
print("Finding ORFs on forward strand:")
orfs_plus = find_orfs_bacterial(genome_sequence)

print("\nFinding ORFs on reverse strand:")
orfs_minus = find_orfs_bacterial(reverse_complement(genome_sequence))

print(f"\nORF search results:")
print(f"Forward strand:  {len(orfs_plus):,} ORFs")
print(f"Reverse strand:  {len(orfs_minus):,} ORFs")
print(f"Total:           {len(orfs_plus) + len(orfs_minus):,} ORFs")

# Show examples of found ORFs
print("\nExamples of found ORFs (first 5):")
for i, orf in enumerate(orfs_plus[:5]):
    print(f"ORF {i+1}: position {orf['start']}-{orf['end']}, "
          f"frame {orf['frame']}, length {orf['length']} bp")
    print(f"  Sequence: {orf['sequence'][:60]}...")

## PART 9: ORF LENGTH DISTRIBUTION ANALYSIS

Building histograms of found ORF length distributions.

In [ ]:
print("\n=== ORF length distribution analysis ===")

def plot_orf_length_distribution(orfs_list, title="ORF Length Distribution"):
    """
    Plots histogram of ORF length distribution
    """
    if not orfs_list:
        print(f"No ORFs found for: {title}")
        return

    orf_lengths = [orf['length'] for orf in orfs_list]

    plt.figure(figsize=(12, 6))
    plt.hist(orf_lengths, bins=50, edgecolor='black', alpha=0.7)
    plt.title(title)
    plt.xlabel("ORF length (nucleotides)")
    plt.ylabel("Number of ORFs")
    plt.yscale('log')  # Log scale for better visualization
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Statistics
    min_length = min(orf_lengths)
    max_length = max(orf_lengths)
    avg_length = sum(orf_lengths) / len(orf_lengths)
    
    print(f"Total number of ORFs: {len(orf_lengths):,}")
    print(f"Shortest ORF: {min_length} bp")
    print(f"Longest ORF: {max_length} bp")
    print(f"Average ORF length: {avg_length:.1f} bp")

# Analyze length distributions
plot_orf_length_distribution(orfs_plus, "ORF Length Distribution (forward strand)")
plot_orf_length_distribution(orfs_minus, "ORF Length Distribution (reverse strand)")

## PART 10: EXTRACTING REFERENCE GENES FROM GFF

Extracting real gene sequences for comparison with our predictions.

In [ ]:
print("\n=== Extracting reference genes from GFF ===")

# Load BioPython if not available
try:
    from Bio import SeqIO
except ImportError:
    print("Installing BioPython...")
    !pip install biopython
    from Bio import SeqIO

import csv

def extract_orfs_from_gff(gff_file, fasta_file):
    """
    Extracts gene sequences from GFF annotation and FASTA file
    """
    # Load sequences
    seq_dict = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))
    orfs = []

    with open(gff_file, 'r') as f:
        reader = csv.reader(f, delimiter='\t')
        for row in reader:
            if len(row) < 9 or row[0].startswith('#') or row[2] != 'gene':
                continue
                
            chrom = row[0]
            start = int(row[3]) - 1  # Convert to 0-based coordinates
            end = int(row[4])
            strand = row[6]

            # Extract sequence
            sequence = seq_dict[chrom].seq[start:end]
            if strand == '-':
                continue  # For now analyze only forward strand
                
            orfs.append({
                'start': start,
                'end': end,
                'frame': (start) % 3,  # Simplified frame determination
                'sequence': str(sequence),
                'length': end - start
            })

    return orfs

# Extract reference genes
print("Extracting reference genes from annotation...")
gff_file = "GCF_000005845.2_ASM584v2_genomic.gff"
fasta_file = "GCF_000005845.2_ASM584v2_genomic.fna"
reference_orfs = extract_orfs_from_gff(gff_file, fasta_file)

print(f"Extracted {len(reference_orfs)} reference genes")
print(f"Predicted {len(orfs_plus)} ORFs")

# Show example reference gene
if reference_orfs:
    print(f"\nExample reference gene:")
    ref_gene = reference_orfs[0]
    print(f"Position: {ref_gene['start']}-{ref_gene['end']}")
    print(f"Length: {ref_gene['length']} bp")
    print(f"Sequence: {ref_gene['sequence'][:60]}...")

## PART 11: PREDICTION QUALITY EVALUATION

For prediction quality evaluation we use standard metrics:

- **True Positive (TP)** - correctly predicted genes
- **False Positive (FP)** - falsely predicted genes (predicted but no gene exists)
- **False Negative (FN)** - missed genes (gene exists but not predicted)

**Metrics:**
- **Precision** = TP / (TP + FP) - fraction of correct among predicted
- **Recall** = TP / (TP + FN) - fraction found among all real
- **F1-score** = 2 * (Precision * Recall) / (Precision + Recall) - harmonic mean

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

def match_orfs(predicted, truth, tolerance=0):
    """
    Matches predicted ORFs with reference ones
    
    Args:
        predicted: list of predicted ORFs
        truth: list of reference ORFs
        tolerance: allowable coordinate deviation
    
    Returns:
        tuple: (TP, FP, FN)
    """
    pred_intervals = [(orf['start'], orf['end']) for orf in predicted]
    true_intervals = [(orf['start'], orf['end']) for orf in truth]
    
    matched_true = set()
    tp = 0  # True Positives
    fp = 0  # False Positives

    # Check each predicted ORF
    for p_start, p_end in pred_intervals:
        found_match = False
        for idx, (t_start, t_end) in enumerate(true_intervals):
            if idx in matched_true:
                continue
            
            # Check coordinate match with tolerance
            if (abs(p_start - t_start) <= tolerance and 
                abs(p_end - t_end) <= tolerance):
                tp += 1
                matched_true.add(idx)
                found_match = True
                break
        
        if not found_match:
            fp += 1

    fn = len(true_intervals) - len(matched_true)  # False Negatives
    return tp, fp, fn

def compute_metrics(tp, fp, fn):
    """Computes quality metrics"""
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
    accuracy = tp / (tp + fp + fn) if (tp + fp + fn) > 0 else 0.0
    
    return {
        'TP': tp,
        'FP': fp,
        'FN': fn,
        'Accuracy': round(accuracy, 4),
        'Precision': round(precision, 4),
        'Recall': round(recall, 4),
        'F1 Score': round(f1, 4),
    }

def evaluate_prediction(predicted_orfs, reference_orfs, tolerance=0):
    """Evaluates prediction quality and prints results"""
    tp, fp, fn = match_orfs(predicted_orfs, reference_orfs, tolerance)
    metrics = compute_metrics(tp, fp, fn)
    
    print(f"Evaluation results (tolerance ±{tolerance} bp):")
    print("-" * 50)
    for metric, value in metrics.items():
        print(f"{metric:12}: {value}")
    
    print(f"\nInterpretation:")
    print(f"- From {len(reference_orfs)} real genes found {tp} ({metrics['Recall']*100:.1f}%)")
    print(f"- From {len(predicted_orfs)} predicted {tp} correct ({metrics['Precision']*100:.1f}%)")
    print(f"- Missed {fn} genes")
    print(f"- Falsely predicted {fp} genes")
    
    return metrics

# Evaluate basic prediction
print("Evaluating basic ORF finding algorithm:")
metrics_base = evaluate_prediction(orfs_plus, reference_orfs, tolerance=0)

## PART 12: FILTERING ORFs BY LENGTH

Most found ORFs are very short and unlikely to code for proteins. Let's apply a minimum length filter.

In [ ]:
print("\n=== Filtering ORFs by length ===")
print("Most found ORFs are very short and unlikely to code for proteins")
print("Applying filter: keep only ORFs longer than 150 nucleotides (50 amino acids)")

# Filter short ORFs
min_length = 150  # nucleotides
orfs_filtered = [orf for orf in orfs_plus if orf["length"] >= min_length]

print(f"Before filtering: {len(orfs_plus):,} ORFs")
print(f"After filtering: {len(orfs_filtered):,} ORFs")
print(f"Filtered out: {len(orfs_plus) - len(orfs_filtered):,} short ORFs")

# Evaluation after filtering
print("\nEvaluation after length filtering:")
metrics_filtered = evaluate_prediction(orfs_filtered, reference_orfs, tolerance=0)

# Compare results
print(f"\nResults comparison:")
print(f"{'Metric':<12} {'Basic':<10} {'Filtered':<12} {'Change'}")
print("-" * 50)
for metric in ['Precision', 'Recall', 'F1 Score']:
    base_val = metrics_base[metric]
    filt_val = metrics_filtered[metric]
    change = filt_val - base_val
    change_str = f"{change:+.3f}"
    print(f"{metric:<12} {base_val:<10.3f} {filt_val:<12.3f} {change_str}")

## PART 13: PREDICTION ERROR ANALYSIS

Detailed analysis of where our algorithm makes mistakes.

In [ ]:
print("\n=== Prediction error analysis ===")

def analyze_prediction_errors(predicted, reference, tolerance=0):
    """
    Detailed analysis of where our algorithm makes mistakes
    """
    # Classify each predicted ORF
    matched_ref = set()
    classified_pred = []
    
    for pred_orf in predicted:
        match_type = 'FP'  # Default False Positive
        
        for idx, ref_orf in enumerate(reference):
            if idx in matched_ref:
                continue
                
            # Check different types of matches
            start_match = abs(pred_orf['start'] - ref_orf['start']) <= tolerance
            end_match = abs(pred_orf['end'] - ref_orf['end']) <= tolerance
            
            if start_match and end_match:
                match_type = 'TP-full'  # Full match
                matched_ref.add(idx)
                break
            elif start_match:
                match_type = 'TP-start'  # Only start matches
                matched_ref.add(idx)
                break
            elif end_match:
                match_type = 'TP-end'  # Only end matches
                matched_ref.add(idx)
                break
        
        classified_pred.append({**pred_orf, 'type': match_type})
    
    # Classify reference ORFs
    classified_ref = []
    for idx, ref_orf in enumerate(reference):
        ref_type = 'FN' if idx not in matched_ref else 'TP'
        classified_ref.append({**ref_orf, 'type': ref_type})
    
    return classified_pred, classified_ref

# Analyze errors
print("Classifying predicted ORFs...")
pred_classified, ref_classified = analyze_prediction_errors(
    orfs_filtered, reference_orfs, tolerance=0
)

# Count match types
from collections import Counter
pred_types = Counter([orf['type'] for orf in pred_classified])

print("Classification results:")
print(f"  Full matches (TP-full):       {pred_types['TP-full']}")
print(f"  Start matches:                {pred_types['TP-start']}")
print(f"  End matches:                  {pred_types['TP-end']}")
print(f"  False predictions (FP):       {pred_types['FP']}")

fn_count = len([orf for orf in ref_classified if orf['type'] == 'FN'])
print(f"  Missed genes (FN):            {fn_count}")

## PART 14: GC CONTENT ANALYSIS

GC content can help distinguish real genes from random ORFs.

In [ ]:
print("\n=== GC content analysis ===")
print("GC content can help distinguish real genes from random ORFs")

def gc_content(sequence):
    """Calculates GC content of sequence (in percent)"""
    sequence = sequence.upper()
    gc_count = sequence.count('G') + sequence.count('C')
    return 100 * gc_count / len(sequence) if len(sequence) > 0 else 0

def compare_gc_content(predicted_classified):
    """Compares GC content of correct and false predictions"""
    tp_gc = []
    fp_gc = []
    
    for orf in predicted_classified:
        gc = gc_content(orf['sequence'])
        if orf['type'] == 'TP-full':
            tp_gc.append(gc)
        elif orf['type'] == 'FP':
            fp_gc.append(gc)
    
    if tp_gc and fp_gc:
        avg_tp = sum(tp_gc) / len(tp_gc)
        avg_fp = sum(fp_gc) / len(fp_gc)
        
        print(f"Average GC content:")
        print(f"  Correct predictions: {avg_tp:.1f}% (n={len(tp_gc)})")
        print(f"  False predictions:   {avg_fp:.1f}% (n={len(fp_gc)})")
        
        # Plot histogram
        plt.figure(figsize=(10, 6))
        plt.hist(tp_gc, bins=20, alpha=0.7, label='Correct (TP)', density=True)
        plt.hist(fp_gc, bins=20, alpha=0.7, label='False (FP)', density=True)
        plt.xlabel('GC content (%)')
        plt.ylabel('Density')
        plt.title('GC Content Distribution: Correct vs False Predictions')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        return avg_tp, avg_fp
    else:
        print("Insufficient data for GC content analysis")
        return None, None

# Analyze GC content
tp_gc, fp_gc = compare_gc_content(pred_classified)

## CONCLUSION

The analysis results show the importance of a comprehensive approach to gene prediction and the necessity of using multiple criteria to distinguish real genes from random open reading frames.

### Key findings:
1. **High level of false predictions** - most ORFs are not genes
2. **Simple search by start/stop codons is insufficient**
3. **Additional filters needed** (GC content, Shine-Dalgarno, codon bias)

### Possible improvements:
1. Search for Shine-Dalgarno sequences before ATG
2. Codon bias analysis
3. Machine learning on sequence features
4. Comparative genomics
5. Expression analysis (RNA-seq data)

In [ ]:
print("\n" + "="*60)
print("CONCLUSION")
print("="*60)

print(f"""
E.coli genome analysis results:

📊 GENOME STATISTICS:
  • Genome length: {len(genome_sequence):,} nucleotides
  • Reference genes: {len(reference_orfs)}
  • Found potential ORFs: {len(orfs_plus):,}
  • After length filtering: {len(orfs_filtered):,}

🎯 PREDICTION QUALITY:
  • Precision: {metrics_filtered['Precision']:.1%}
  • Recall: {metrics_filtered['Recall']:.1%}
  • F1-score: {metrics_filtered['F1 Score']:.3f}

🔍 MAIN PROBLEMS:
  1. High level of false predictions - most ORFs are not genes
  2. Simple start/stop codon search is insufficient
  3. Additional filters needed (GC content, Shine-Dalgarno, codon bias)

💡 POSSIBLE IMPROVEMENTS:
  1. Search for Shine-Dalgarno sequences before ATG
  2. Codon bias analysis
  3. Machine learning on sequence features
  4. Comparative genomics
  5. Expression analysis (RNA-seq data)

This analysis shows the importance of a comprehensive approach to gene prediction
and the necessity of using multiple criteria to distinguish real genes
from random open reading frames.
""")

print("\nAdjust filtering parameters and try other approaches!")
print("Good luck studying bioinformatics! 🧬")